# ValueSvc
Provides an FS API for non-container values such as `str`, `int`, and `bool`.

In [ ]:
from fs.base import FS
from fs.info import Info
from fs.enums import ResourceType
from fs.errors import DirectoryExpected
from fs.errors import ResourceInvalid
from fs.errors import ResourceReadOnly
from io import BytesIO
from sys import getsizeof

class ValueSvc(FS):
    
    def __init__(self, value):
        self.__value = value
        
    # ---- pyfilesystem API ----
        
    def getinfo(self, path, namespaces=['basic']):
        """Get info about the value.
        
        The most relevant info will be in the details namespace.
        
        If this function uses sys.getsizeof, it will return the
        memory footprint of the python object, not the length
        in bytes of the data of interest.
        """
        info = {
            'basic': {
                'name': str(type(self.__value)),
                'is_dir': False
            }
        }
        if 'details' in (namespaces if namespaces else []):
            details = {
                'size': getsizeof(self.__value)
            }
            info.update(details)
        return Info(info)
    
    def listdir(self, _):
        raise DirectoryExpected('ValueSvc value does not have a directory.')
    
    def makedir(self, _):
        raise ResourceInvalid('ValueSvc cannot make a directory inside a value.')
    
    def openbin(self, _, *args, **kwargs):
        return BytesIO(str(self.__value).encode())
    
    def remove(self, _):
        raise ResourceInvalid('ValueSvc cannot have directories inside a value. Nothing to remove.')
    
    def removedir(self, _):
        raise ResourceInvalid('ValueSvc cannot have directories inside a value. Nothing to remove.')

    def setinfo(self, _):
        raise ResourceReadOnly('ValueSvc cannot set arbitrary info.')
    
    def isdir(self, _):
        """ValueSvc::isdir() unconditionally returns false."""
        return False
    
    def writetext(self, _, value):
        self.__value = value
    
    # ---- Dcel API ----
    
    @property
    def value(self):
        """Returns the internal value."""
        return self.__value
    
    @value.setter
    def set_value(self, value):
        """Sets the internal value."""
        self.__value = value
    

In [61]:
def ValueSvcTests(values, newvalues, methods):
    
    def test__init__(value):
        return ValueSvc(value)
    def test_getinfo(obj,*args,**kwargs):
        return obj.getinfo('')
    
    # run
    for value,newvalue in zip(values,newvalues):
        v = ValueSvc(value)
        for method in methods:
            try:
                print (f"Testing {method} on {value}...")
                m = getattr(v,method)
                res = m(
                  *(methods[method]['args']),
                  **(methods[method]['kwargs'])
                 )
                print(f"Result of {method}: {res}")
            except Exception as e:
                print(e)
        # special writetext
        try:
            print (f"Testing writetext on {value}...")
            res = v.writetext('.',newvalue)
            print(f"Result of writetext on {value}: {res}")
        except Exception as e:
            print(e)
        try:
            print(f"Testing readtext on new value of {value}...")
            res = v.readtext('.')
            print(f"Result of readtext on new value of {value}: {res}")
        except Exception as e:
            print(e)
        print('-------------------------------------------')
    

In [63]:
# Setup test
values = ["hello",1,True]
newvalues = ["konichiwa",2,False]
methods = {
    'getinfo': {'args':['.'],'kwargs':{'namespaces':['details']}},
    'listdir': {'args':['.'],'kwargs':{}},
    'makedir': {'args':['abcd'],'kwargs':{}},
    'openbin': {'args':['.'],'kwargs':{}},
    'remove': {'args':['abcd'],'kwargs':{}},
    'removedir': {'args':['abcd'],'kwargs':{}},
    # 'writetext': {'args':['.','konichiwa'],'kwargs':{}},
    'readtext': {'args':['.'],'kwargs':{}}
}

In [64]:
ValueSvcTests(values,newvalues,methods)

Testing getinfo on hello...
Result of getinfo: <file '<class 'str'>'>
Testing listdir on hello...
path 'ValueSvc value does not have a directory.' should be a directory
Testing makedir on hello...
resource 'ValueSvc cannot make a directory inside a value.' is invalid for this operation
Testing openbin on hello...
Result of openbin: <_io.BytesIO object at 0x7faf88235ad0>
Testing remove on hello...
resource 'ValueSvc cannot have directories inside a value. Nothing to remove.' is invalid for this operation
Testing removedir on hello...
resource 'ValueSvc cannot have directories inside a value. Nothing to remove.' is invalid for this operation
Testing readtext on hello...
Result of readtext: hello
Testing writetext on hello...
Result of writetext on hello: None
Testing readtext on new value of hello...
Result of readtext on new value of hello: konichiwa
-------------------------------------------
Testing getinfo on 1...
Result of getinfo: <file '<class 'int'>'>
Testing listdir on 1...
path

In [33]:
for method in methods:
    print(method)
    print(methods[method])

getinfo
{'args': ['.'], 'kwargs': {'namespaces': ['details']}}
listdir
{'args': ['.']}
makedir
{'args': ['abcd']}
openbin
{'args': ['.']}
remove
{'args': ['abcd']}
removedir
{'args': ['abcd']}
writetext
{'args': ['.', 'konichiwa']}
readtext
{'args': ['.']}


In [ ]:
ValueSvcTest(values,newvalues,methods)

In [22]:
# TEST __init__()
sfs = ValueSvc("hello")

In [23]:
# TEST getinfo()
sfs.getinfo('.', namespaces=['details']).raw

{'basic': {'name': 'str', 'is_dir': False}, 'size': 54}

In [24]:
# TEST value()
sfs.value

'hello'

In [25]:
# TEST openbin
sfs.openbin('.')

In [26]:
## Secondary pyfilesystem tests
# These should automatically be inferred by pyfilesystem.

In [27]:
# Test readtext
sfs.readtext('.')

'hello'

In [7]:
# Test gettext
sfs.gettext('.')

'hello'

In [12]:
sfs.writetext('.','konichiwa')

In [13]:
sfs.gettext('.')

'konichiwa'